In [1]:
import os

In [2]:
%pwd

'e:\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("E:\\Kidney-Disease-Classification-Deep-Learning-Project")

In [4]:
%pwd

'E:\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'saikishan2452003'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '0410d70c0b85ca93d146bdbf64e9ca2b448d2e8d'

In [6]:
import tensorflow as tf



In [7]:
import os
print(f"Current directory: {os.getcwd()}")
print(f"\nContents of current directory:")
print(os.listdir("."))
print(f"\nSearching for model files...")
for root, dirs, files in os.walk("."):
    for file in files:
        if "model" in file.lower() and file.endswith((".h5", ".hdf5", ".keras")):
            print(os.path.join(root, file))

Current directory: E:\Kidney-Disease-Classification-Deep-Learning-Project

Contents of current directory:
['.conda', '.git', '.gitattributes', '.github', '.gitignore', 'artifacts', 'config', 'dvc.yaml', 'logs', 'main.py', 'mlruns', 'params.yaml', 'README.md', 'requirements.txt', 'research', 'scores.json', 'setup.py', 'src', 'template.py', 'templates']

Searching for model files...
.\artifacts\prepare_base_model\base_model.h5
.\artifacts\prepare_base_model\base_model_updated.h5
.\artifacts\training\model.h5
.\mlruns\952858982974295664\models\m-7bb7d65c04f04508bd21b8ecdb268f05\artifacts\data\model.keras
.\mlruns\952858982974295664\models\m-7f23132b12c14457bde1dcf26da14c26\artifacts\data\model.keras
.\mlruns\952858982974295664\models\m-b4a498fe76904b10b7f4dd79cd2d1bb1\artifacts\data\model.keras


In [8]:
import os
print(f"Current dir: {os.getcwd()}")
print(f"Artifacts dir exists: {os.path.exists('artifacts')}")
if os.path.exists('artifacts/training'):
    print(f"Files in artifacts/training: {os.listdir('artifacts/training/')}")
else:
    print("artifacts/training directory does not exist")
    
# Try to load the model
model = tf.keras.models.load_model("artifacts/training/model.h5")

Current dir: E:\Kidney-Disease-Classification-Deep-Learning-Project
Artifacts dir exists: True
Files in artifacts/training: ['model.h5']


In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
from pathlib import Path
import mlflow
# Remove: import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            # Log parameters
            mlflow.log_params(self.config.all_params)
            
            # Log metrics
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Log dataset information
            mlflow.log_param("dataset_path", str(self.config.training_data))
            mlflow.log_param("validation_split", 0.30)
            
            # Log description as a tag
            mlflow.set_tag("description", "Model evaluation on kidney CT scan images")
            mlflow.set_tag("task", "binary_classification")
            mlflow.set_tag("framework", "TensorFlow/Keras")
            
            # Log model artifact
            mlflow.keras.log_model(self.model, "model")
            
            # Log model registry if not file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            
            # Log model metadata
            mlflow.log_param("model_path", str(self.config.path_of_model))
            mlflow.log_param("model_type", "VGG16")

In [15]:
import os
# Move up one level from 'research' to the project root
os.chdir("../") 
print(f"Current working directory: {os.getcwd()}")

Current working directory: E:\


In [16]:
import os
os.chdir("E:\\Kidney-Disease-Classification-Deep-Learning-Project")
print(f"Current dir: {os.getcwd()}")
print(f"Artifacts dir exists: {os.path.exists('artifacts')}")
if os.path.exists('artifacts/training'):
    print(f"Files in artifacts/training: {os.listdir('artifacts/training/')}")
else:
    print("artifacts/training directory does not exist")

Current dir: E:\Kidney-Disease-Classification-Deep-Learning-Project
Artifacts dir exists: True
Files in artifacts/training: ['model.h5']


In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    print("Model evaluation completed successfully!")
    print(f"Loss: {evaluation.score[0]}")
    print(f"Accuracy: {evaluation.score[1]}")
    
    # MLflow logging (optional - skip if connection issues)
    # evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4820 - loss: 229.3513
Model evaluation completed successfully!
Loss: 229.351318359375
Accuracy: 0.48201438784599304


In [19]:
import mlflow
import dagshub
import os
from datetime import datetime

# Initialize DagHub MLflow tracking
dagshub.init(repo_owner='saikishan2452003', 
             repo_name='Kidney-Disease-Classification-Deep-Learning-Project',
             mlflow=True)

# Explicitly set the MLflow tracking URI to DagHub remote
mlflow.set_tracking_uri('https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow')

try:
    # Set experiment name
    mlflow.set_experiment("kidney-disease")
    
    with mlflow.start_run(run_name="evaluation"):
        # Log metrics
        mlflow.log_metric("loss", float(evaluation.score[0]))
        mlflow.log_metric("accuracy", float(evaluation.score[1]))
        
        # Log dataset information - MLflow specific format for Dataset column
        dataset_info = {
            "name": "kidney-ct-scan-image",
            "path": "artifacts/data_ingestion/kidney-ct-scan-image",
            "hash": "kidney-disease-v1",
            "profile": "CT scan images for kidney disease classification"
        }
        mlflow.log_param("dataset_name", dataset_info["name"])
        mlflow.log_param("dataset_path", dataset_info["path"])
        mlflow.log_param("dataset_hash", dataset_info["hash"])
        
        # Set tags for UI display - these appear as columns
        mlflow.set_tag("mlflow.datasets.name", "kidney-ct-scan-image")
        mlflow.set_tag("mlflow.datasets.hash", "kidney-disease-v1")
        mlflow.set_tag("mlflow.datasets.uri", "artifacts/data_ingestion/kidney-ct-scan-image")
        
        # Set version tag
        mlflow.set_tag("version", "1.0")
        
        # Set description tag - this should show in Description column
        mlflow.set_tag("mlflow.runDescription", "VGG16 Model evaluation on kidney CT scan images for disease classification")
        mlflow.set_tag("description", "VGG16 Model evaluation on kidney CT scan images for disease classification")
        
        # Log other parameters
        mlflow.log_param("validation_split", 0.30)
        mlflow.log_param("image_size", str(eval_config.params_image_size))
        mlflow.log_param("batch_size", eval_config.params_batch_size)
        mlflow.log_param("model_name", "VGG16")
        mlflow.log_param("model_path", "artifacts/training/model.h5")
        mlflow.log_param("model_version", "1.0")
        
        # Set additional tags for metadata
        mlflow.set_tag("task", "binary_classification")
        mlflow.set_tag("framework", "TensorFlow/Keras")
        mlflow.set_tag("evaluation_date", datetime.now().strftime("%Y-%m-%d"))
        
        # Log model artifact
        mlflow.keras.log_model(evaluation.model, "model")
        
        print("✓ Metrics logged to DagHub MLflow successfully!")
        print(f"  Dataset: kidney-ct-scan-image")
        print(f"  Version: 1.0")
        print(f"  Model: VGG16")
        print(f"  Description: VGG16 Model evaluation on kidney CT scan images")
        print(f"  Loss: {evaluation.score[0]:.4f}")
        print(f"  Accuracy: {evaluation.score[1]:.4f}")
        print(f"\n  View on DagHub: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project/experiments")
        
except Exception as mlflow_error:
    print(f"✗ DagHub MLflow logging failed: {mlflow_error}")
    print(f"  Error details: {str(mlflow_error)}")


Initialized MLflow to track repo "saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project"

Repository saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project initialized!

2026/01/21 00:34:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/21 00:34:34 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


✓ Metrics logged to DagHub MLflow successfully!
  Dataset: kidney-ct-scan-image
  Version: 1.0
  Model: VGG16
  Description: VGG16 Model evaluation on kidney CT scan images
  Loss: 229.3513
  Accuracy: 0.4820

  View on DagHub: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project/experiments
🏃 View run evaluation at: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/b68520ddef7c4f8e809f679052c55fbc
🧪 View experiment at: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0


In [ ]:
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [25]:
import mlflow
import dagshub
import os

# Check current MLflow configuration
print("=== MLflow Configuration Check ===\n")

# Initialize DagHub
dagshub.init(repo_owner='saikishan2452003', 
             repo_name='Kidney-Disease-Classification-Deep-Learning-Project',
             mlflow=True)

# Check tracking URI
tracking_uri = mlflow.get_tracking_uri()
print(f"MLflow Tracking URI: {tracking_uri}")

# Check environment variables
print(f"\nMLFLOW_TRACKING_URI: {os.environ.get('MLFLOW_TRACKING_URI', 'NOT SET')}")
print(f"MLFLOW_TRACKING_USERNAME: {os.environ.get('MLFLOW_TRACKING_USERNAME', 'NOT SET')}")
print(f"MLFLOW_TRACKING_PASSWORD: {'*' * 20 if os.environ.get('MLFLOW_TRACKING_PASSWORD') else 'NOT SET'}")

# List experiments
print(f"\n=== Existing Experiments ===")
experiments = mlflow.search_experiments()
for exp in experiments:
    print(f"  - {exp.name} (ID: {exp.experiment_id})")

=== MLflow Configuration Check ===



Initialized MLflow to track repo "saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project"

Repository saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project initialized!

MLflow Tracking URI: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow

MLFLOW_TRACKING_URI: https://dagshub.com/saikishan2452003/Kidney-Disease-Classification-Deep-Learning-Project.mlflow
MLFLOW_TRACKING_USERNAME: saikishan2452003
MLFLOW_TRACKING_PASSWORD: ********************

=== Existing Experiments ===
  - kidney-disease (ID: 0)
